In [56]:
from PIL import Image
import os
import numpy as np
from tqdm import tqdm
from colormath.color_diff import delta_e_cie1976
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color


In [57]:
import cv2
import numpy as np
import os
from sklearn.cluster import KMeans
from collections import Counter
from tqdm import tqdm
import pandas as pd

# Define the path to the "Camera1/front" folder
front_folder = "Camera1/front"

# Function to get the top 10 colors using k-means clustering
def get_top_colors(image, n_clusters=10):
    # Convert the image to RGB color space
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Reshape the image to be a list of pixels
    pixels = rgb_image.reshape(-1, 3)

    # Fit the k-means clustering model
    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(pixels)

    # Get the cluster centers (dominant colors)
    centers = kmeans.cluster_centers_

    # Count the pixels assigned to each cluster
    counts = Counter(kmeans.labels_)

    # Get the most common colors
    top_colors = [centers[i] for i, _ in counts.most_common(n_clusters)]

    return top_colors

# Initialize lists to store image names and top colors
image_names = []
top_colors_list = []

# Get the list of image files
image_files = os.listdir(front_folder)

# Iterate over each image file in the folder with progress tracking
for filename in tqdm(image_files, desc="Processing images"):
    # Construct the full path to the image file
    filepath = os.path.join(front_folder, filename)

    # Load the image and resize it to one-fourth of its original size
    image = cv2.imread(filepath)
    resized_image = cv2.resize(image, (0, 0), fx=0.25, fy=0.25)

    # Get the top 10 colors in the resized image
    top_colors = get_top_colors(resized_image, n_clusters=10)

    # Append image name to the list
    image_names.append(filename.split(".")[0])

    # Append top colors to the list
    top_colors_list.append(top_colors)

# Create a DataFrame
data = {"ID": image_names, "Color ID": top_colors_list}
df = pd.DataFrame(data)


Processing images:   0%|                                                                      | 0/1092 [00:00<?, ?it/s]C:\Users\adminsyma\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
Processing images:   0%|                                                              | 1/1092 [00:03<56:59,  3.13s/it]C:\Users\adminsyma\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
Processing images:   0%|                                                              | 2/1092 [00:05<53:31,  2.95s/it]C:\Users\adminsyma\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. S

In [58]:
df

,ID,Color ID
0,0,"[[29.065440947336363, 34.02166406980294, 7.578..."
1,1,"[[37.04832922967201, 40.47069996481638, 17.660..."
2,10,"[[36.156609454565825, 39.4909880534476, 15.996..."
3,100,"[[32.04973013493261, 37.06592203898384, 17.074..."
4,1000,"[[39.662532530461704, 43.23666931628888, 21.88..."
...,...,...
1087,995,"[[39.48020538079204, 43.10522280563041, 21.381..."
1088,996,"[[34.42572380890006, 39.824831710345194, 16.50..."
1089,997,"[[41.361913221523196, 45.46549336063331, 23.37..."
1090,998,"[[41.76695999325544, 45.30529393573238, 23.891..."


# Getting colors names:

In [59]:
from colorthief import ColorThief

In [60]:
df

,ID,Color ID
0,0,"[[29.065440947336363, 34.02166406980294, 7.578..."
1,1,"[[37.04832922967201, 40.47069996481638, 17.660..."
2,10,"[[36.156609454565825, 39.4909880534476, 15.996..."
3,100,"[[32.04973013493261, 37.06592203898384, 17.074..."
4,1000,"[[39.662532530461704, 43.23666931628888, 21.88..."
...,...,...
1087,995,"[[39.48020538079204, 43.10522280563041, 21.381..."
1088,996,"[[34.42572380890006, 39.824831710345194, 16.50..."
1089,997,"[[41.361913221523196, 45.46549336063331, 23.37..."
1090,998,"[[41.76695999325544, 45.30529393573238, 23.891..."


## Round numbers

In [61]:
# Function to round the numbers in the Color ID list
def round_numbers(color_list):
    return [(round(r), round(g), round(b)) for (r, g, b) in color_list]

# Round the numbers in the Color ID column
df["Color ID"] = df["Color ID"].apply(round_numbers)

In [62]:
df["Color ID"][1]

[(37, 40, 18),
 (45, 47, 27),
 (28, 33, 10),
 (224, 234, 183),
 (56, 59, 38),
 (70, 73, 49),
 (207, 218, 169),
 (188, 195, 149),
 (163, 168, 127),
 (136, 137, 102)]

In [63]:
# Dictionary of known colors and their RGB values
known_colors = {
    "aliceblue": (240, 248, 255),
    "antiquewhite": (250, 235, 215),
    "aqua": (0, 255, 255),
    "aquamarine": (127, 255, 212),
    "azure": (240, 255, 255),
    "beige": (245, 245, 220),
    "bisque": (255, 228, 196),
    "black": (0, 0, 0),
    "blanchedalmond": (255, 235, 205),
    "blue": (0, 0, 255),
    "blueviolet": (138, 43, 226),
    "brown": (165, 42, 42),
    "burlywood": (222, 184, 135),
    "cadetblue": (95, 158, 160),
    "chartreuse": (127, 255, 0),
    "chocolate": (210, 105, 30),
    "coral": (255, 127, 80),
    "cornflowerblue": (100, 149, 237),
    "cornsilk": (255, 248, 220),
    "crimson": (220, 20, 60),
    "cyan": (0, 255, 255),
    "darkblue": (0, 0, 139),
    "darkcyan": (0, 139, 139),
    "darkgoldenrod": (184, 134, 11),
    "darkgray": (169, 169, 169),
    "darkgreen": (0, 100, 0),
    "darkgrey": (169, 169, 169),
    "darkkhaki": (189, 183, 107),
    "darkmagenta": (139, 0, 139),
    "darkolivegreen": (85, 107, 47),
    "darkorange": (255, 140, 0),
    "darkorchid": (153, 50, 204),
    "darkred": (139, 0, 0),
    "darksalmon": (233, 150, 122),
    "darkseagreen": (143, 188, 143),
    "darkslateblue": (72, 61, 139),
    "darkslategray": (47, 79, 79),
    "darkslategrey": (47, 79, 79),
    "darkturquoise": (0, 206, 209),
    "darkviolet": (148, 0, 211),
    "deeppink": (255, 20, 147),
    "deepskyblue": (0, 191, 255),
    "dimgray": (105, 105, 105),
    "dimgrey": (105, 105, 105),
    "dodgerblue": (30, 144, 255),
    "firebrick": (178, 34, 34),
    "floralwhite": (255, 250, 240),
    "forestgreen": (34, 139, 34),
    "fuchsia": (255, 0, 255),
    "gainsboro": (220, 220, 220),
    "ghostwhite": (248, 248, 255),
    "gold": (255, 215, 0),
    "goldenrod": (218, 165, 32),
    "gray": (128, 128, 128),
    "green": (0, 128, 0),
    "greenyellow": (173, 255, 47),
    "grey": (128, 128, 128),
    "honeydew": (240, 255, 240),
    "hotpink": (255, 105, 180),
    "indianred": (205, 92, 92),
    "indigo": (75, 0, 130),
    "ivory": (255, 255, 240),
    "khaki": (240, 230, 140),
    "lavender": (230, 230, 250),
    "lavenderblush": (255, 240, 245),
    "lawngreen": (124, 252, 0),
    "lemonchiffon": (255, 250, 205),
    "lightblue": (173, 216, 230),
    "lightcoral": (240, 128, 128),
    "lightcyan": (224, 255, 255),
    "lightgoldenrodyellow": (250, 250, 210),
    "lightgray": (211, 211, 211),
    "lightgreen": (144, 238, 144),
    "lightgrey": (211, 211, 211),
    "lightpink": (255, 182, 193),
    "lightsalmon": (255, 160, 122),
    "lightseagreen": (32, 178, 170),
    "lightslategray": (119, 136, 153),
    "lightslategrey": (119, 136, 153),
    "lightsteelblue": (176, 196, 222),
    "lightyellow": (255, 255, 224),
    "lime": (0, 255, 0),
    "limegreen": (50, 205, 50),
    "linen": (250, 240, 230),
    "magenta": (255, 0, 255),
    "maroon": (128, 0, 0),
    "mediumaquamarine": (102, 205, 170),
    "mediumblue": (0, 0, 205),
    "mediumorchid": (186, 85, 211),
    "mediumpurple": (147, 112, 219),
    "mediumseagreen": (60, 179, 113),
    "mediumslateblue": (123, 104, 238),
    "mediumspringgreen": (0, 250, 154),
    "mediumturquoise": (72, 209, 204),
    "mediumvioletred": (199, 21, 133),
    "midnightblue": (25, 25, 112),
    "mintcream": (245, 255, 250),
    "mistyrose": (255, 228, 225),
    "moccasin": (255, 228, 181),
    "navajowhite": (255, 222, 173),
    "navy": (0, 0, 128),
    "oldlace": (253, 245, 230),
    "olive": (128, 128, 0),
    "olivedrab": (107, 142, 35),
    "orange": (255, 165, 0),
    "orangered": (255, 69, 0),
    "orchid": (218, 112, 214),
    "palegoldenrod": (238, 232, 170),
    "palegreen": (152, 251, 152),
    "paleturquoise": (175, 238, 238),
    "palevioletred": (219, 112, 147),
    "papayawhip": (255, 239, 213),
    "peachpuff": (255, 218, 185),
    "peru": (205, 133, 63),
    "pink": (255, 192, 203),
    "plum": (221, 160, 221),
    "powderblue": (176, 224, 230),
    "purple": (128, 0, 128),
    "rebeccapurple": (102, 51, 153),
    "red": (255, 0, 0),
    "rosybrown": (188, 143, 143),
    "royalblue": (65, 105, 225),
    "saddlebrown": (139, 69, 19),
    "salmon": (250, 128, 114),
    "sandybrown": (244, 164, 96),
    "seagreen": (46, 139, 87),
    "seashell": (255, 245, 238),
    "sienna": (160, 82, 45),
    "silver": (192, 192, 192),
    "skyblue": (135, 206, 235),
    "slateblue": (106, 90, 205),
    "slategray": (112, 128, 144),
    "slategrey": (112, 128, 144),
    "snow": (255, 250, 250),
    "springgreen": (0, 255, 127),
    "steelblue": (70, 130, 180),
    "tan": (210, 180, 140),
    "teal": (0, 128, 128),
    "thistle": (216, 191, 216),
    "tomato": (255, 99, 71),
    "turquoise": (64, 224, 208),
    "violet": (238, 130, 238),
    "wheat": (245, 222, 179),
    "white": (255, 255, 255),
    "whitesmoke": (245, 245, 245),
    "yellow": (255, 255, 0),
    "yellowgreen": (154, 205, 50),
    "Red": (255, 0, 0),
    "Green": (0, 255, 0),
    "Blue": (0, 0, 255),
    "Yellow": (255, 255, 0),
    "Cyan": (0, 255, 255),
    "Magenta": (255, 0, 255),
    "White": (255, 255, 255),
    "Black": (0, 0, 0)
}

In [64]:
# Function to find the closest color name to the given RGB tuple
def rgb_to_color_name(rgb_color):
    min_distance = float('inf')
    closest_color = None
    
    for color_name, color_rgb in known_colors.items():
        distance = np.linalg.norm(np.array(rgb_color) - np.array(color_rgb))
        if distance < min_distance:
            min_distance = distance
            closest_color = color_name
    
    return closest_color

# Apply the color name conversion to each row in the DataFrame with tqdm
tqdm.pandas(desc="Converting colors")
df["Colors"] = df["Color ID"].progress_apply(lambda colors: [rgb_to_color_name(color) for color in colors])

Converting colors: 100%|███████████████████████████████████████████████████████████| 1092/1092 [00:11<00:00, 93.82it/s]


In [65]:
# Convert the 'ID' column to integer
df['ID'] = df['ID'].astype(int)
# Reorder the DataFrame based on the 'ID' column
df = df.sort_values(by='ID')
# Reindex the DataFrame
df = df.reset_index(drop=True)
df

,ID,Color ID,Colors
0,0,"[(29, 34, 8), (30, 35, 15), (37, 40, 15), (23,...","[black, black, black, black, darkslategray, le..."
1,1,"[(37, 40, 18), (45, 47, 27), (28, 33, 10), (22...","[black, darkslategray, black, palegoldenrod, d..."
2,2,"[(253, 177, 2), (36, 38, 12), (47, 43, 17), (2...","[orange, black, black, black, darkolivegreen, ..."
3,3,"[(34, 38, 14), (27, 32, 8), (41, 43, 21), (212...","[black, black, black, palegoldenrod, darkslate..."
4,4,"[(33, 36, 13), (26, 31, 7), (40, 41, 19), (201...","[black, black, black, firebrick, black, darksl..."
...,...,...,...
1087,1096,"[(44, 48, 28), (38, 42, 24), (42, 45, 18), (33...","[darkslategray, black, black, black, darkslate..."
1088,1097,"[(42, 45, 26), (36, 40, 21), (46, 49, 29), (42...","[darkslategray, black, darkslategray, black, b..."
1089,1098,"[(40, 44, 25), (39, 56, 60), (34, 39, 16), (48...","[black, darkslategray, black, darkslategray, d..."
1090,1099,"[(38, 42, 21), (44, 47, 24), (33, 39, 14), (42...","[black, darkslategray, black, darkslategray, d..."


In [66]:
df.loc[38]

ID                                                         40
Color ID    [(31, 36, 11), (37, 40, 17), (25, 31, 7), (46,...
Colors      [black, black, black, darkslategray, darkseagr...
Name: 38, dtype: object

In [67]:
df

,ID,Color ID,Colors
0,0,"[(29, 34, 8), (30, 35, 15), (37, 40, 15), (23,...","[black, black, black, black, darkslategray, le..."
1,1,"[(37, 40, 18), (45, 47, 27), (28, 33, 10), (22...","[black, darkslategray, black, palegoldenrod, d..."
2,2,"[(253, 177, 2), (36, 38, 12), (47, 43, 17), (2...","[orange, black, black, black, darkolivegreen, ..."
3,3,"[(34, 38, 14), (27, 32, 8), (41, 43, 21), (212...","[black, black, black, palegoldenrod, darkslate..."
4,4,"[(33, 36, 13), (26, 31, 7), (40, 41, 19), (201...","[black, black, black, firebrick, black, darksl..."
...,...,...,...
1087,1096,"[(44, 48, 28), (38, 42, 24), (42, 45, 18), (33...","[darkslategray, black, black, black, darkslate..."
1088,1097,"[(42, 45, 26), (36, 40, 21), (46, 49, 29), (42...","[darkslategray, black, darkslategray, black, b..."
1089,1098,"[(40, 44, 25), (39, 56, 60), (34, 39, 16), (48...","[black, darkslategray, black, darkslategray, d..."
1090,1099,"[(38, 42, 21), (44, 47, 24), (33, 39, 14), (42...","[black, darkslategray, black, darkslategray, d..."


In [68]:
# Remove black from the 'colors' column in df
df['Colors'] = df['Colors'].apply(lambda x: [color for color in x if color.lower() != 'black'])

df

,ID,Color ID,Colors
0,0,"[(29, 34, 8), (30, 35, 15), (37, 40, 15), (23,...","[darkslategray, lemonchiffon, darkolivegreen, ..."
1,1,"[(37, 40, 18), (45, 47, 27), (28, 33, 10), (22...","[darkslategray, palegoldenrod, darkslategray, ..."
2,2,"[(253, 177, 2), (36, 38, 12), (47, 43, 17), (2...","[orange, darkolivegreen, orange, darkgoldenrod..."
3,3,"[(34, 38, 14), (27, 32, 8), (41, 43, 21), (212...","[palegoldenrod, darkslategray, silver, darksea..."
4,4,"[(33, 36, 13), (26, 31, 7), (40, 41, 19), (201...","[firebrick, darkslategray, saddlebrown, darkre..."
...,...,...,...
1087,1096,"[(44, 48, 28), (38, 42, 24), (42, 45, 18), (33...","[darkslategray, darkslategray, darkslategray, ..."
1088,1097,"[(42, 45, 26), (36, 40, 21), (46, 49, 29), (42...","[darkslategray, darkslategray, darkslategray, ..."
1089,1098,"[(40, 44, 25), (39, 56, 60), (34, 39, 16), (48...","[darkslategray, darkslategray, darkslategray, ..."
1090,1099,"[(38, 42, 21), (44, 47, 24), (33, 39, 14), (42...","[darkslategray, darkslategray, darkslategray, ..."


In [69]:
# Remove black from the 'colors' column in df
df['Colors'] = df['Colors'].apply(lambda x: [color for color in x if color.lower() != 'darkslategray'])

df

,ID,Color ID,Colors
0,0,"[(29, 34, 8), (30, 35, 15), (37, 40, 15), (23,...","[lemonchiffon, darkolivegreen, palegoldenrod, ..."
1,1,"[(37, 40, 18), (45, 47, 27), (28, 33, 10), (22...","[palegoldenrod, darkolivegreen, palegoldenrod,..."
2,2,"[(253, 177, 2), (36, 38, 12), (47, 43, 17), (2...","[orange, darkolivegreen, orange, darkgoldenrod..."
3,3,"[(34, 38, 14), (27, 32, 8), (41, 43, 21), (212...","[palegoldenrod, silver, darkseagreen, mediumaq..."
4,4,"[(33, 36, 13), (26, 31, 7), (40, 41, 19), (201...","[firebrick, saddlebrown, darkred, maroon, sienna]"
...,...,...,...
1087,1096,"[(44, 48, 28), (38, 42, 24), (42, 45, 18), (33...","[darkolivegreen, dimgray, rosybrown]"
1088,1097,"[(42, 45, 26), (36, 40, 21), (46, 49, 29), (42...","[darkolivegreen, gray]"
1089,1098,"[(40, 44, 25), (39, 56, 60), (34, 39, 16), (48...","[midnightblue, gray]"
1090,1099,"[(38, 42, 21), (44, 47, 24), (33, 39, 14), (42...","[darkolivegreen, dimgray, gray]"


In [70]:
for colors in df["Colors"]:
    print(colors)

['lemonchiffon', 'darkolivegreen', 'palegoldenrod', 'darkkhaki']
['palegoldenrod', 'darkolivegreen', 'palegoldenrod', 'tan', 'darkseagreen', 'gray']
['orange', 'darkolivegreen', 'orange', 'darkgoldenrod', 'sienna', 'sienna', 'darkgoldenrod']
['palegoldenrod', 'silver', 'darkseagreen', 'mediumaquamarine', 'dimgray', 'lightseagreen']
['firebrick', 'saddlebrown', 'darkred', 'maroon', 'sienna']
['gray', 'dimgray']
['darkolivegreen', 'dimgray', 'dimgray']
['dimgray', 'darkseagreen']
['darkcyan', 'teal', 'seagreen', 'midnightblue', 'slategray']
['darkolivegreen', 'darkolivegreen', 'sienna', 'gray']
['dimgray', 'dimgray', 'gray']
['dimgray', 'gray', 'darkseagreen', 'palegoldenrod']
['slategray']
['lightgoldenrodyellow', 'palegoldenrod', 'palegoldenrod', 'darkolivegreen', 'tan', 'gray']
['silver', 'tan', 'darkseagreen', 'darkolivegreen', 'darkkhaki', 'olivedrab']
['dimgray', 'darkolivegreen', 'darkolivegreen', 'gray', 'darkseagreen', 'darkgray', 'palegoldenrod']
['saddlebrown', 'maroon', 'dark

In [74]:
from collections import Counter

# Concatenate all color lists into one list
all_colors = [color for color_list in df['Colors'] for color in color_list]

# Count the occurrences of each color
color_counter = Counter(all_colors)

# Get the most common 10 colors
most_common_colors = color_counter.most_common(20)

print("The 10 most frequent colors in all of the data are:")
for color, frequency in most_common_colors:
    print(f"Color: {color}, Frequency: {frequency}")


The 10 most frequent colors in all of the data are:
Color: dimgray, Frequency: 784
Color: gray, Frequency: 536
Color: darkolivegreen, Frequency: 525
Color: darkseagreen, Frequency: 258
Color: silver, Frequency: 254
Color: tan, Frequency: 226
Color: palegoldenrod, Frequency: 188
Color: seagreen, Frequency: 119
Color: slategray, Frequency: 87
Color: darkgray, Frequency: 84
Color: darkkhaki, Frequency: 67
Color: darkslateblue, Frequency: 56
Color: lightgoldenrodyellow, Frequency: 51
Color: saddlebrown, Frequency: 38
Color: cadetblue, Frequency: 36
Color: rosybrown, Frequency: 32
Color: teal, Frequency: 27
Color: firebrick, Frequency: 26
Color: darkgoldenrod, Frequency: 24
Color: sienna, Frequency: 22


In [75]:
# Define the colors to remove
colors_to_remove = ['dimgray', 'gray', 'darkolivegreen', 'darkseagreen', 'silver',
                    'tan', 'palegoldenrod', 'seagreen', 'slategray', 'darkgray']

# Remove the specified colors from the 'Colors' column in df
df['Colors'] = df['Colors'].apply(lambda x: [color for color in x if color.lower() not in colors_to_remove])


In [76]:
df

,ID,Color ID,Colors
0,0,"[(29, 34, 8), (30, 35, 15), (37, 40, 15), (23,...","[lemonchiffon, darkkhaki]"
1,1,"[(37, 40, 18), (45, 47, 27), (28, 33, 10), (22...",[]
2,2,"[(253, 177, 2), (36, 38, 12), (47, 43, 17), (2...","[orange, orange, darkgoldenrod, sienna, sienna..."
3,3,"[(34, 38, 14), (27, 32, 8), (41, 43, 21), (212...","[mediumaquamarine, lightseagreen]"
4,4,"[(33, 36, 13), (26, 31, 7), (40, 41, 19), (201...","[firebrick, saddlebrown, darkred, maroon, sienna]"
...,...,...,...
1087,1096,"[(44, 48, 28), (38, 42, 24), (42, 45, 18), (33...",[rosybrown]
1088,1097,"[(42, 45, 26), (36, 40, 21), (46, 49, 29), (42...",[]
1089,1098,"[(40, 44, 25), (39, 56, 60), (34, 39, 16), (48...",[midnightblue]
1090,1099,"[(38, 42, 21), (44, 47, 24), (33, 39, 14), (42...",[]


In [86]:
# Fill empty lists in the 'Colors' column with "Small/Transparent"
df['Colors'] = df['Colors'].apply(lambda x: ["Small/Transparent"] if not x else x)
df

,ID,Color ID,Colors
0,0,"[(29, 34, 8), (30, 35, 15), (37, 40, 15), (23,...","[lemonchiffon, darkkhaki]"
1,1,"[(37, 40, 18), (45, 47, 27), (28, 33, 10), (22...",[Small/Transparent]
2,2,"[(253, 177, 2), (36, 38, 12), (47, 43, 17), (2...","[orange, orange, darkgoldenrod, sienna, sienna..."
3,3,"[(34, 38, 14), (27, 32, 8), (41, 43, 21), (212...","[mediumaquamarine, lightseagreen]"
4,4,"[(33, 36, 13), (26, 31, 7), (40, 41, 19), (201...","[firebrick, saddlebrown, darkred, maroon, sienna]"
...,...,...,...
1087,1096,"[(44, 48, 28), (38, 42, 24), (42, 45, 18), (33...",[rosybrown]
1088,1097,"[(42, 45, 26), (36, 40, 21), (46, 49, 29), (42...",[Small/Transparent]
1089,1098,"[(40, 44, 25), (39, 56, 60), (34, 39, 16), (48...",[midnightblue]
1090,1099,"[(38, 42, 21), (44, 47, 24), (33, 39, 14), (42...",[Small/Transparent]


In [87]:
for index, row in df.iterrows():
    print(f"Index: ",row["ID"],"Colors:", row["Colors"])


Index:  0 Colors: ['lemonchiffon', 'darkkhaki']
Index:  1 Colors: ['Small/Transparent']
Index:  2 Colors: ['orange', 'orange', 'darkgoldenrod', 'sienna', 'sienna', 'darkgoldenrod']
Index:  3 Colors: ['mediumaquamarine', 'lightseagreen']
Index:  4 Colors: ['firebrick', 'saddlebrown', 'darkred', 'maroon', 'sienna']
Index:  5 Colors: ['Small/Transparent']
Index:  6 Colors: ['Small/Transparent']
Index:  7 Colors: ['Small/Transparent']
Index:  8 Colors: ['darkcyan', 'teal', 'midnightblue']
Index:  9 Colors: ['sienna']
Index:  10 Colors: ['Small/Transparent']
Index:  11 Colors: ['Small/Transparent']
Index:  12 Colors: ['Small/Transparent']
Index:  13 Colors: ['lightgoldenrodyellow']
Index:  15 Colors: ['darkkhaki', 'olivedrab']
Index:  16 Colors: ['Small/Transparent']
Index:  17 Colors: ['saddlebrown', 'maroon']
Index:  18 Colors: ['gold', 'goldenrod', 'darkgoldenrod']
Index:  19 Colors: ['skyblue', 'mediumaquamarine']
Index:  20 Colors: ['lightgray', 'darkslateblue', 'lightslategray']
Index

In [88]:
# Save the DataFrame to a CSV file
df.to_csv('color_data.csv', index=False)